##  Image Classification for OxfordIIITPet

In [1]:
# Install dependencies
!pip install torchmetrics torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 8.1 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchinfo import  summary
from torchmetrics import Accuracy
from tqdm.auto import tqdm
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np

torch.__version__, torchvision.__version__

('2.0.1+cu118', '0.15.2+cu118')

In [3]:
# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

### 1. Create data transforms
Create transform with normalization to match the data distribution of the EfficientNetB0

Alternative approach:
```
# 1. Setup pretrained EffNetB0 weights
effnetb0_weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT

# 2. Get EffNetB0 transforms
effnetb0_transforms = effnetb2_weights.transforms()
```

In [ ]:
# Create EffnetB0 model data transform
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.TrivialAugmentWide(num_magnitude_bins=31), # how intense the transformation
    transforms.ToTensor()
    # normalize
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

### Get the OxfordIIITPet dataset

In [ ]:
# Load train and test datasets
train_dataset = datasets.OxfordIIITPet(root="data",
                                       split="trainval",
                                       transform=train_transforms,
                                       download=True);

test_dataset = datasets.OxfordIIITPet(root="data",
                                     split="test",
                                     transform=test_transforms,
                                     download=True);

class_names = train_dataset.classes
class_names_idx = train_dataset.class_to_idx

### Select a subset for experimentation

In [ ]:
# Define a list of class indices you want to keep (0-based)
selected_classes = [0, 1, 2]  # Select the first three classes

# Filter the dataset to keep only the specified classes
train_dataset = [item for item in train_dataset if item[1] in selected_classes]
test_dataset = [item for item in train_dataset if item[1] in selected_classes]


In [ ]:
print(f"Train dataset number of samples: {len(train_dataset)}")
print(f"Test dataset number of samples: {len(test_dataset)}")


Train dataset number of samples: 300
Test dataset number of samples: 300


In [ ]:
# # Check for imbalance
# from collections import Counter

# targets = [train_dataset[i][1] for i in np.arange(0, len(train_dataset))]
# class_counts = Counter(targets)
# print(f"Class counts: {class_counts}")

### Visualize images

In [ ]:
nrows = 3
ncols = 4

random.seed(13)
# Select random images from train dataset
random_images = []
random_labels = []
for idx in random.sample(list(np.arange(0, len(train_dataset))), k=nrows*ncols):
    random_images.append(train_dataset[idx][0])
    random_labels.append(train_dataset[idx][1])

# Plot random images with labels
plt.figure(figsize=(9, 10))

for i, sample_image in enumerate(random_images):
    plt.subplot(nrows, ncols, i+1)
    plt.imshow(sample_image.permute(1, 2, 0))
    plt.title(class_names[random_labels[i]])
    plt.axis(False)

### Create Dataloaders

In [ ]:
# Set batch_size and num_workers
BATCH_SIZE = 32
NUM_WORKERS = 2

train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=NUM_WORKERS)

test_dataloader = DataLoader(dataset=test_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=False,
                              num_workers=NUM_WORKERS)

# Print number of batches
print(f"Train DataLoader number of batches: {len(train_dataloader)}")
print(f"Test DataLoader number of batches: {len(test_dataloader)}")
print(f"Batch shape: {next(iter(train_dataloader))[0].shape}") # (batch_size, color_channels, height, width)

Train DataLoader number of batches: 10
Test DataLoader number of batches: 10
Batch shape: torch.Size([32, 3, 224, 224])


### Create EfficientNet_B0

In [ ]:
# Get model weights
effnetb0_weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
# Create a model instance
effnetb0 = torchvision.models.efficientnet_b0(weights=effnetb0_weights)
# Freeze the base layers in the mode0
for param in effnetb0.parameters():
    param.requires_grad = False

In [ ]:
torch.manual_seed(13)
torch.cuda.manual_seed(13)

# Change the classifier layer (trainable)
effnetb0.classifier = nn.Sequential(
    nn.Dropout(p=0.3, inplace=True),
    nn.Linear(in_features=1280, out_features=len(class_names), bias=True)
)

In [ ]:
from torchinfo import summary

summary(effnetb0,
        input_size=(1, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

### Train Model

In [ ]:
# Select optimizer
optimizer = torch.optim.Adam(params=effnetb0.parameters(),
                             lr=1e-3)

# Set loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Set accuracy function
accuracy_fn = Accuracy(task='multiclass', num_classes=len(class_names))

### At this point copy paste plot_functions.py and torchTrain.py from github


In [ ]:
from torchTrain import torchTrain

trainer = torchTrain()


In [ ]:
torch.manual_seed(13)
trainer.train(model=effnetb0,
            train_dataloader=train_dataloader,
            test_dataloader=test_dataloader,
            optimizer=optimizer,
            loss_fn=loss_fn,
            accuracy_fn=accuracy_fn,
            epochs=5,
            device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0 | Train loss: 0.3614 | Train acc: 0.9344 | Test loss: 0.3214 | Test acc: 0.9615
Epoch: 1 | Train loss: 0.3107 | Train acc: 0.9656 | Test loss: 0.2855 | Test acc: 0.9906
Epoch: 2 | Train loss: 0.2544 | Train acc: 0.9635 | Test loss: 0.2508 | Test acc: 0.9906
Epoch: 3 | Train loss: 0.2577 | Train acc: 0.9573 | Test loss: 0.2222 | Test acc: 0.9906
Epoch: 4 | Train loss: 0.2356 | Train acc: 0.9625 | Test loss: 0.1975 | Test acc: 0.9969


{'train_loss': [0.36140695214271545,
  0.31070247292518616,
  0.25438016653060913,
  0.25773507356643677,
  0.23563659191131592],
 'train_acc': [0.934374988079071,
  0.965624988079071,
  0.9635416865348816,
  0.9572917222976685,
  0.9624999761581421],
 'test_loss': [0.3214138150215149,
  0.28553181886672974,
  0.2507922649383545,
  0.2221616506576538,
  0.19753167033195496],
 'test_acc': [0.9614583253860474,
  0.9906250238418579,
  0.9906250238418579,
  0.9906250238418579,
  0.996874988079071]}

In [ ]:
from plot_functions import  plot_loss_curves

plot_loss_curves(results)

In [ ]:
def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory.

    Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

    Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory if it doesn't exist
    os.makedirs(target_dir, exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = os.path.join(target_dir, model_name)

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(), f=model_save_path)


def load_model(model: torch.nn.Module, model_path: str):
    """Loads a PyTorch model from a specified model path.

    Args:
    model: An instance of the PyTorch model to load the state_dict into.
    model_path: The path to the saved model file.

    Example usage:
    loaded_model = load_model(model=my_model, model_path="models/my_model.pth")
    """
    # Load the model state_dict
    model.load_state_dict(torch.load(model_path))

    # Ensure the model is in evaluation mode
    model.eval()

    print(f"[INFO] Loaded model from: {model_path}")

    return model


In [ ]:
import os
save_model(effnetb0,
           'models',
           'test_model.pt')

[INFO] Saving model to: models/test_model.pt
